In [1]:
import pandas as pd

In [2]:
#Leitura dos dados do contrato
df1 = pd.read_csv('Teste casa dos ventos/contracts.csv', sep=';')
#Leitura dos dados da garantia física
df2 = pd.read_csv('Teste casa dos ventos/gf.csv')
#Leitura dos dados para cálculo da liquidacao esperada
df3 = pd.read_csv('Teste casa dos ventos/geracao_futura.csv')
#Leitura dos preços forward
df4 = pd.read_csv('Teste casa dos ventos/price_forward.csv')

In [3]:
df1['volume_mwh'] = df1['volume_mwm']*(31*24)
df1.rename(columns = {'party' : 'park'}, inplace = True)
pd.set_option('display.max_rows', 144)

In [4]:
df_filtro = df1.groupby(['park', 'month', 'type', 'energy_type']).agg({'volume_mwh':'sum'}).reset_index()
df_compra1 = df_filtro[((df_filtro['type'] == 'compra')&(df_filtro['month'] >= 7))]
df_venda1 = df_filtro[((df_filtro['type'] == 'venda')&(df_filtro['month'] >= 7))]
df_compra = df_compra1.merge(df4[['month','price_ne']], how = 'left', on = ['month'])
df_venda = df_venda1.merge(df4[['month','price_ne']], how = 'left', on = ['month'])
df_compra['valor'] = df_compra['price_ne']*df_compra['volume_mwh']
df_venda['valor'] = df_venda['price_ne']*df_venda['volume_mwh']

In [5]:
df_geral = df_venda.merge(df_compra, how = 'left', on = ['park', 'month', 'energy_type', 'price_ne'], suffixes=('_venda', '_compra'))
del df_geral['type_compra']
del df_geral['type_venda']
df_geral['volume_mwh_compra'] = df_geral['volume_mwh_compra'].fillna(0)
df_geral['valor_compra'] = df_geral['valor_compra'].fillna(0)

In [6]:
#Transformando os dados da garantia física
df2['gf_mwh'] = df2['gf_mwm']*(31*24)
del df2['gf_mwm']

In [7]:
#Adicionando os dados da garantia física ao dataframe geral
df_geral = df_geral.merge(df2, how = 'left', on = ['park', 'month'])                                                

In [8]:
#Cálculo do balanço de garantia física
df_geral['balanco_gf'] = df_geral['gf_mwh'] + df_geral['volume_mwh_compra'] - df_geral['volume_mwh_venda']

In [9]:
#Transformando os dados da geração futura
df3['p50_mwh'] = df3['p50_mwm']*(31*24)
del df3['p50_mwm']

In [10]:
#Adicionando os dados da geração futura ao dataframe geral
df_geral = df_geral.merge(df3, how = 'left', on = ['park', 'month'])
del df_geral['complex']
del df_geral['p90_mwm']
df_geral.rename(columns = {'p50_mwh' : 'geracao_prevista_mwh'}, inplace = True)

In [11]:
#Cálculo do balanço energético esperado
df_geral['balanco_energetico_esperado'] = df_geral['geracao_prevista_mwh'] + df_geral['volume_mwh_compra'] - df_geral['volume_mwh_venda']

In [12]:
#Adicionando o spread ao dataframe geral
df_geral = df_geral.merge(df4[['month','spread_incentivada']], how = 'left', on = ['month'])
df_geral.loc[df_geral['energy_type']=='convencional', 'spread_incentivada'] = 0

In [13]:
#Cálculo das grandezas requeridas
df_geral['liquidacao_esperada'] = df_geral['balanco_energetico_esperado']*df_geral['price_ne']
df_geral['receita_esperada'] = df_geral['valor_venda'] - df_geral['valor_compra'] + df_geral['liquidacao_esperada'] + df_geral['balanco_gf']*df_geral['spread_incentivada']
df_geral['receita_contratos'] = df_geral['valor_venda'] - df_geral['valor_compra']
df_geral['receita_exposicao'] = df_geral['geracao_prevista_mwh']*df_geral['price_ne']

In [14]:
#Arquivos de saída
df_geral.to_csv('parte2.csv')
df_geral.to_excel('parte2.xlsx')